<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/DataModelAntoineEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064_Fall2021/main/WaterVaporPressure.csv

--2021-08-23 22:44:27--  https://raw.githubusercontent.com/profteachkids/CHE2064_Fall2021/main/WaterVaporPressure.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [text/plain]
Saving to: ‘WaterVaporPressure.csv’

WaterVaporPressure. 100%[===================>]   1.27K  --.-KB/s    in 0s      

2021-08-23 22:44:27 (108 MB/s) - ‘WaterVaporPressure.csv’ saved [1300/1300]



In [14]:
from lxml import html
import requests
import jax
import jax.numpy as jnp
import numpy as np
from scipy.optimize import minimize
from plotly.subplots import make_subplots
import plotly.io as pio
from collections import deque
pio.templates.default='plotly_dark'

#JAX default is 32 bit - enable 64 bit - double precision
from jax.config import config
config.update("jax_enable_x64", True)

In [16]:
data=np.loadtxt("WaterVaporPressure.csv", delimiter=",")

In [21]:
T=data[:,0]
log_Pv = np.log10(data[:,1])

In [22]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=T, y=log_Pv, mode='markers', marker_size=8, marker_color='rgba(0,0,0,0)',
                marker_line_color='rgb(0,0,255)', marker_line_width=1, name='Data')
fig.update_layout(width=800, height=500)
fig.update_layout(xaxis_title='T (K)', yaxis_title='$Log_{10} P\ (Pa)$a')
fig.show()

In [23]:
def model(a, b, c, T):
  return a-b/(T+c)

In [30]:
a_guess = 10.
b_guess = 2000.0
c_guess = 0.0
x0 = [a_guess, b_guess, c_guess]

def sqerror(x, T, log_Pv):
  model_Pv = model(x[0],x[1],x[2], T)
  err = jnp.sum( (model_Pv-log_Pv)**2)
  return err

res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10)
#res = minimize(sqerror, x0=x0, args=(T, log_Pv), tol=1e-10, jac=jax.grad(sqerror))

In [28]:
res

      fun: 0.0005381098250991544
 hess_inv: array([[6.05908056e+00, 4.73256607e+03, 4.85712128e+02],
       [4.73256607e+03, 3.74063396e+06, 3.87679374e+05],
       [4.85712128e+02, 3.87679374e+05, 4.06142595e+04]])
      jac: DeviceArray([-6.44817533e-12, -1.78376872e-14,  2.87241307e-13], dtype=float64)
  message: 'Optimization terminated successfully.'
     nfev: 48
      nit: 42
     njev: 48
   status: 0
  success: True
        x: array([  10.20869185, 1756.35645945,  -36.02769483])

In [29]:
T_model = jnp.linspace(T[0],T[-1],100)
fig.add_scatter(x=T_model, y=model(*res.x, T_model), mode='lines', line_width=1, name='Antoine')
fig.show()